In [1]:
#import streamlit as st
#import pandas as pd
#import altair as alt
#import plotly.express as px

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from datetime import datetime
import matplotlib.dates as mdates
import matplotlib.patches as mpatches
import matplotlib.lines as mlines
import matplotlib.ticker as mtick
import os
import platform
import glob

In [3]:
if platform.system()=='Windows': 
    #Directorios Windows
    os.chdir(
        r"G:\.shortcut-targets-by-id\120bUa35y3FJzvumWahpOx1qS9I6TIrhb\GIGANAV")

if platform.system()=='Darwin':
    os.chdir(
        '/Users/juliandiazparra/Library/CloudStorage/GoogleDrive-julian.adiaz@hotmail.com/Mi unidad/Negocios/Familiar/GIGANAV')

folder=os.getcwd()
data=os.path.join(folder,'Datos')
provisiones=os.path.join(data,'Provisiones')
facturas=os.path.join(data,'Facturas')
tickets=os.path.join(data,'Tickets')
usuarios=os.path.join(data,'Usuarios')
egresos=os.path.join(data,'Egresos')
transacciones=os.path.join(data,'Transacciones')
analisis=os.path.join(folder,'Operacion/Analisis')
resultados=os.path.join(analisis,'Resultados')
bases=os.path.join(analisis,'Bases_analisis')
outputs=os.path.join(analisis,'Outputs')
plots=os.path.join(outputs,'Gráficos')
usuarios_clean=os.path.join(bases,'Usuarios')
transac_clean=os.path.join(bases,'Transacciones')
tickets_clean=os.path.join(bases,'Tickets')
facturas_clean=os.path.join(bases,'Facturas')
egresos_clean=os.path.join(bases,'Egresos')
working=os.path.join(bases,'Bases_trabajo')
codigo=os.path.join(folder,'Operacion/Analisis/Código')
conciliaciones=os.path.join(folder,'Operacion/Operativo/Administrativo/Conciliaciones')

## Importación base de trabajo

In [4]:
list_of_files = glob.glob(os.path.join(outputs,'Resumen_mes_categorias*.csv'))
latest_file = max(list_of_files, key=os.path.getctime)
working_summary=pd.read_csv(latest_file, low_memory=False)
working_summary.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34 entries, 0 to 33
Data columns (total 62 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   Unnamed: 0                                34 non-null     int64  
 1   mes_crd                                   34 non-null     object 
 2   tiempo_respuesta_Instalacion_fibra        33 non-null     float64
 3   tiempo_respuesta_Instalacion_radio        34 non-null     float64
 4   tiempo_respuesta_Retiro_fibra             34 non-null     float64
 5   tiempo_respuesta_Retiro_radio             34 non-null     float64
 6   tiempo_respuesta_Soporte_fibra            34 non-null     float64
 7   tiempo_respuesta_Soporte_radio            34 non-null     float64
 8   tiempo_respuesta_Traslado_fibra           34 non-null     float64
 9   tiempo_respuesta_Traslado_radio           34 non-null     float64
 10  num_tickets_Instalacion_fibra           

In [5]:
list_of_files = glob.glob(os.path.join(outputs,'Base_tickets*.csv'))
latest_file = max(list_of_files, key=os.path.getctime)
working_data=pd.read_csv(latest_file, low_memory=False)
working_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16080 entries, 0 to 16079
Data columns (total 70 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             16080 non-null  int64  
 1   id_ticket              16080 non-null  int64  
 2   departamento           16080 non-null  object 
 3   nombre                 16080 non-null  object 
 4   asunto                 16080 non-null  object 
 5   tecnico                16080 non-null  object 
 6   zona                   16040 non-null  object 
 7   abierto_por            16080 non-null  object 
 8   instal                 16080 non-null  int64  
 9   soporte                16080 non-null  int64  
 10  traslado               16080 non-null  int64  
 11  retiro                 16080 non-null  int64  
 12  mantenimiento          16080 non-null  int64  
 13  facturacion            16080 non-null  int64  
 14  cambio_plan            16080 non-null  int64  
 15  su

## Panel resumen Coordinador

In [6]:
#Hago resumen Coordinador
working_summary.loc[
    :,'per_encuestas'
]=working_summary.loc[
    :,['per_enc_Instalacion',
       'per_enc_Soporte']].mean(axis=1)

In [7]:
#Indicadores
valor_variable=380000
indicadores_tecnicos=pd.DataFrame()
indicadores_tecnicos.loc[
    :,'Indicadores']=['tiempo_respuesta_Soporte_fibra',
        'tiempo_respuesta_Instalacion_fibra', 
        'per_encuestas']
indicadores_tecnicos.loc[
    :,'Peso'
]=[.4,.4,.2]
indicadores_tecnicos.loc[
    :,'Valor'
]=indicadores_tecnicos[
    'Peso']*valor_variable

indicadores_tecnicos

,Indicadores,Peso,Valor
0,tiempo_respuesta_Soporte_fibra,0.4,152000.0
1,tiempo_respuesta_Instalacion_fibra,0.4,152000.0
2,per_encuestas,0.2,76000.0


In [8]:
mes=list(working_summary.loc[pd.to_datetime(
    working_summary['mes_crd']).dt.strftime(
    '%Y')==datetime.today().strftime(
    '%Y'), 'mes_crd'].unique())

filepath=os.path.join(
    outputs,
    'Indicadores_coordinador_'+datetime.today().strftime('%Y')+'.xlsx')

writer = pd.ExcelWriter(filepath, 
                        engine = 'xlsxwriter') 

for m in mes:
    resumen_coordinador=pd.DataFrame()
    resumen_coordinador.loc[
        :,m
    ]=working_summary.loc[
        working_summary['mes_crd']==m, [
            'mes_crd', 
            'tiempo_respuesta_Soporte_fibra',
            'tiempo_respuesta_Instalacion_fibra', 
            'per_encuestas'
        ]
    ].iloc[-1].iloc[1:]

    resumen_coordinador.loc[
        :,'Meta'
    ]=[3.5,3.5,.7]

    resumen_coordinador.loc[
        :,'% cumplimiento'
    ]=1-((resumen_coordinador[
        m]-resumen_coordinador[
        'Meta'])/resumen_coordinador[
        'Meta'])

    resumen_coordinador.loc[
        resumen_coordinador.index=='per_encuestas', 
        '% cumplimiento']=resumen_coordinador[
        m]/resumen_coordinador['Meta']

    resumen_coordinador.reset_index(inplace=True)
    resumen_coordinador.rename(
        columns={'index':'Indicadores'}, inplace=True)
    resumen_coordinador=resumen_coordinador.merge(
        indicadores_tecnicos[['Indicadores','Valor']], 
        on='Indicadores')

    #Creo compensaciones
    resumen_coordinador.loc[
        :,'Compensacion']=np.select(
        [(resumen_coordinador['% cumplimiento']<.8)] ,[0],
        default=(resumen_coordinador[
            '% cumplimiento']*resumen_coordinador[
            'Valor']))
    
    resumen_coordinador.to_excel(
                writer, sheet_name=m, index=False)
    
writer.close()

In [10]:
working_summary[['mes_crd','tiempo_respuesta_Instalacion_fibra',
                 'tiempo_respuesta_Soporte_fibra','per_encuestas']].tail()

,mes_crd,tiempo_respuesta_Instalacion_fibra,tiempo_respuesta_Soporte_fibra,per_encuestas
29,2024-06,4.241618,5.284792,0.688525
30,2024-07,4.952676,5.473163,0.444388
31,2024-08,3.374170,2.282151,0.447016
32,2024-09,2.627447,1.807004,0.595691
33,2024-10,NaN,1.740564,0.899718
